In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import plot_histogram

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [10]:
q = QuantumRegister(6, 'q')
c = ClassicalRegister(3, 'c')
c_qftadder = QuantumCircuit(q, c)

#Test
c_qftadder.x(q[2])
#c_qftadder.x(q[1])
#c_qftadder.x(q[0])


#QFT
c_qftadder.barrier(q[0],q[1],q[2],q[3],q[4],q[5])

c_qftadder.h(q[5])
c_qftadder.cp(np.pi/4,q[3],q[5])
c_qftadder.cp(np.pi/2,q[4],q[5])
c_qftadder.h(q[4])
c_qftadder.cp(np.pi/2,q[3],q[4])
c_qftadder.h(q[3])
c_qftadder.swap(q[3], q[5])

c_qftadder.barrier(q[0],q[1],q[2],q[3],q[4],q[5])

#CZ
c_qftadder.cp(np.pi, q[0], q[3])
c_qftadder.cp(np.pi / 2, q[1], q[3])
c_qftadder.cp(np.pi / 4, q[2], q[3])
c_qftadder.cp(np.pi, q[1], q[4])
c_qftadder.cp(np.pi / 2, q[2], q[4])
c_qftadder.cp(np.pi, q[2], q[5])

c_qftadder.barrier(q[0],q[1],q[2],q[3],q[4],q[5])

#IQFT
c_qftadder.swap(q[3], q[5])
c_qftadder.h([3])
c_qftadder.cp(-np.pi/2, q[3], q[4])
c_qftadder.h(q[4])
c_qftadder.cp(-np.pi/2, q[4], q[5])
c_qftadder.cp(-np.pi/4, q[3], q[5])
c_qftadder.h(q[5])

#Measure
c_qftadder.barrier(q[0],q[1],q[2],q[3],q[4],q[5])
c_qftadder.measure(q[3], c[0])
c_qftadder.measure(q[4], c[1])
c_qftadder.measure(q[5], c[2])

ibmqfactory.load_account:WARNING:2022-06-29 01:26:48,917: Credentials are already in use. The existing account in the session will be replaced.



Total count for 00 and 11 are: {'000': 861, '001': 155, '010': 208, '011': 299, '100': 1325, '101': 410, '110': 401, '111': 341}


OrderedDict([('cx', 60),
             ('rz', 42),
             ('sx', 12),
             ('x', 7),
             ('barrier', 4),
             ('measure', 3)])

In [11]:
%matplotlib inline
c_qftadder.draw(output='mpl')

In [18]:
#provider = IBMQ.get_provider(hub='ibm-q-skku', group='korea-university', project='ku-graduate')
#backend = provider.get_backend('ibmq_qasm_simulator')
#backend = provider.get_backend('ibm_perth')
simulator = Aer.get_backend('aer_simulator')

In [19]:
re_transpile=transpile(c_qftadder, backend)
#re_transpile.draw(output='mpl', idle_wires=False)

In [20]:
re_transpile=transpile(c_qftadder, backend, optimization_level = 3)
#re_transpile.draw(output='mpl', idle_wires=False)

In [21]:
#job = execute(c_qftadder, backend, shots=100000)
#job = execute(re_transpile, backend)
#result = job.result()
#counts = result.get_counts(c_qftadder)
result=simulator.run(re_transpile).result()
counts = result.get_counts(re_transpile)
print("\nTotal count for 00 and 11 are:",counts)
#_qftadder.draw()
re_transpile.draw()

In [ ]:
re_transpile.count_ops()